In [2]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 10)
source('tests.R')
source("cleanup.R")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [72]:
#Kaggle requires an APIs for access but the document itself is free for public use
#It allows us to directly download it from the website so we downloaded the file then loaded it into R

salary_region<-read_csv("data/salaries_by_region.csv")

salary_region <- salary_region |>
    rename_with(~str_to_lower(.) |>              #reformat the column names into lower case and replace space with underscores
                str_replace_all("[^[:alnum:]]+", "_"), .cols = everything())|>
    select(school_name:mid_career_median_salary)
salary_region

Rows: 320 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): School Name, Region, Mid-Career 10th Percentile Salary, Mid-Career ...
dbl (4): Starting Median Salary, Mid-Career Median Salary, Mid-Career 25th P...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


school_name,region,starting_median_salary,mid_career_median_salary
<chr>,<chr>,<dbl>,<dbl>
Stanford University,California,70400,129000
California Institute of Technology (CIT),California,75500,123000
Harvey Mudd College,California,71800,122000
University of California Berkeley,California,59900,112000
Occidental College,California,51900,105000
⋮,⋮,⋮,⋮
State University of New York (SUNY) at Potsdam,Northeastern,38000,70300
Niagara University,Northeastern,36900,69700
State University of New York (SUNY) at Fredonia,Northeastern,37800,66200
